In [1]:
from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.topology.molecule import Molecule
from openforcefield.topology.topology import Topology

from system.typing.smirnoff import build_smirnoff_map, build_smirnoff_collection
from system.collections import PotentialHandler, PotentialCollection
from system.system import System
from system.utils import get_test_file_path

In [2]:
# Load in a minimal SMIRNOFF forcefield and Argon topology
argon_ff = ForceField(get_test_file_path('ar.offxml'))

mol = Molecule.from_smiles('[#18]')
mol.generate_conformers(n_conformers=1)

argon_top =  Topology.from_molecules(10 * [mol])

In [3]:
# Generate a mapping of topology slots and SMIRNOFF SMIRKS
smirks_map = build_smirnoff_map(forcefield=argon_ff, topology=argon_top)
# {
#   handler_name1: {
#     {
#       slot1: SMIRKS1,
#       slot2: SMIRKS2,
#       slot3: SMIRKS3,
#       ...,
#       slotN: SMIRKSN,
#     },
#   handler_name2: { ... },
#   handler_name3: { ... },
#   ...
#   handler_nameN: { ... },
# }
smirks_map

{'vdW': {(0,): '[#18:1]',
  (1,): '[#18:1]',
  (2,): '[#18:1]',
  (3,): '[#18:1]',
  (4,): '[#18:1]',
  (5,): '[#18:1]',
  (6,): '[#18:1]',
  (7,): '[#18:1]',
  (8,): '[#18:1]',
  (9,): '[#18:1]'}}

In [4]:
smirnoff_collection = build_smirnoff_collection(forcefield=argon_ff)

In [5]:
argon_system = System(
    potential_collection=smirnoff_collection,
    topology=argon_top,
)

In [6]:
# The topology attribute simply stores the toolkit topology
type(argon_system.topology)

openforcefield.topology.topology.Topology

In [7]:
# The potential_collection is effectively a force field for SMIRNOFF systems,
# but is more flexible for future typing schemes that are much less "1:1" than
# traiditional atom-typing or SMIRNOFF, which generally map 1 parameter to a slot
type(argon_system.potential_collection)

system.collections.PotentialCollection

In [8]:
# A PotentialCollection object stores "types" of potentials, i.e. one section for vdW,
# one section for bonds, etc. in order to separate parameters from instructions; defining
# a SMIRKS pattern and a slot is not sufficient instructions to apply the corresponding 
# parameter. This system only has vdW, but this dict is intended to store other non-bonded
# terms, valence terms, wild cross-coupling terms, etc.
argon_system.potential_collection.handlers.keys()

dict_keys(['vdW'])

In [9]:
# A PotentialCollection is more or less nested dictionaries, so you can drill down
# to find the value of a particular parameter in a potential, returning a 
# unit-bearing quantity (via pint).
argon_system.potential_collection['vdW'].potentials['[#18:1]'].parameters['sigma']
# The path is convoluted, but goes something like this
# sys_name.potential_collection['vdW'].potentials[smirks].parameters['sigma']
#    ^                            ^                 ^                   ^
#    |                            |                 |                   | 
#    |                            |                 |                   | 
#    |                "type" (?) of potential       |     finally, the actual param
#    |                                              |
# system I care about                SMIRKS key, mapping to a potential

0.3 <Unit('nanometer')>

In [10]:
# There's other information included in each potential, notably the
# analytical expression of the potential and the SMIRKS
argon_system.potential_collection['vdW'].potentials['[#18:1]']

ParametrizedAnalyticalPotential(name='n1', smirks='[#18:1]', expression='4*epsilon*((sigma/r)**12-(sigma/r)**6)', independent_variables={'r'}, parameters={'sigma': <Quantity(0.3, 'nanometer')>, 'epsilon': <Quantity(0.1, 'kilojoule / mole')>})

In [11]:
# Going back to the smirks_map, which is a dictionary mapping "slots" to potentials
# In the simple case, this is just atom indicies : SMKIRKS
smirks_map['vdW']

{(0,): '[#18:1]',
 (1,): '[#18:1]',
 (2,): '[#18:1]',
 (3,): '[#18:1]',
 (4,): '[#18:1]',
 (5,): '[#18:1]',
 (6,): '[#18:1]',
 (7,): '[#18:1]',
 (8,): '[#18:1]',
 (9,): '[#18:1]'}

In [12]:
# The smirks_map and potential_collection can be used to look up potentials
# using the 'vdW' and SMIRKS as sufficient information
argon_system.potential_collection['vdW'][smirks_map['vdW'][(7,)]]

ParametrizedAnalyticalPotential(name='n1', smirks='[#18:1]', expression='4*epsilon*((sigma/r)**12-(sigma/r)**6)', independent_variables={'r'}, parameters={'sigma': <Quantity(0.3, 'nanometer')>, 'epsilon': <Quantity(0.1, 'kilojoule / mole')>})

In [13]:
# Being built off of pydantic, we get a serializable representation for free,
# caveats being that some components are not actually serializable now
argon_system.dict()

{'topology': <openforcefield.topology.topology.Topology at 0x11f658f50>,
 'potential_collection': {'handlers': {'vdW': {'name': 'vdW',
    'potentials': {'[#18:1]': {'name': 'n1',
      'smirks': '[#18:1]',
      'expression': '4*epsilon*((sigma/r)**12-(sigma/r)**6)',
      'independent_variables': {'r'},
      'parameters': {'sigma': 0.3 <Unit('nanometer')>,
       'epsilon': 0.1 <Unit('kilojoule / mole')>}}}}}},
 'positions': None,
 'box': None,
 'slots_map': None}

In [14]:
# Being a nested dictionary, you could also grab a component and make that a dict
argon_system.potential_collection.handlers['vdW'].potentials['[#18:1]'].dict()

{'name': 'n1',
 'smirks': '[#18:1]',
 'expression': '4*epsilon*((sigma/r)**12-(sigma/r)**6)',
 'independent_variables': {'r'},
 'parameters': {'sigma': 0.3 <Unit('nanometer')>,
  'epsilon': 0.1 <Unit('kilojoule / mole')>}}